# Deep v. shallow sequences

References:

* [Raymond Hettinger's anwswer to "Are tuples more efficient than lists in Python?"](https://stackoverflow.com/questions/68630/are-tuples-more-efficient-than-lists-in-python/22140115#22140115): includes tuple and list structs

* [Python internals: Arbitrary-precision integer implementation](https://rushter.com/blog/python-integer-implementation/), by Artem Golubin: includes introspection of object structures with `ctypes`

## Inspecting a tuple of `float`

Code from Hettinger's answer:

```c
typedef struct {
        Py_ssize_t ob_refcnt;
        struct _typeobject *ob_type;
        Py_ssize_t ob_size;
        PyObject *ob_item[2];     /* store a pointer to 10 and a pointer to 20 */
    } PyTupleObject;
```

From `cpython/Include/floatobject.h` (with expanded macros)

```c
typedef struct {
    Py_ssize_t ob_refcnt;         /* PyObject_HEAD */
    struct _typeobject *ob_type;  /* PyObject_HEAD */
    double ob_fval;
} PyFloatObject;
```

In [1]:
import ctypes

class PyFloatObject(ctypes.Structure):
    _fields_ = [("ob_refcnt", ctypes.c_long),
                ("ob_type", ctypes.c_void_p),
                ("ob_val", ctypes.c_double),
               ]

n = 5.0
float_struct = PyFloatObject.from_address(id(n))

print('ob_refcnt:', float_struct.ob_refcnt)
print('ob_val:', float_struct.ob_val)

ob_refcnt: 2
ob_val: 5.0


In [2]:
t = (1.0, 2.0)

ITEMS = len(t)

class PyTupleObject(ctypes.Structure):
    _fields_ = [("ob_refcnt", ctypes.c_long),
                ("ob_type", ctypes.c_void_p),
                ("ob_size", ctypes.c_ulong),
                ("ob_item", ctypes.c_void_p * ITEMS)]


print('t:', t)
print('item ids:', tuple(id(i) for i in t))

t_struct = PyTupleObject.from_address(id(t))

print('ob_size:', t_struct.ob_size)
print('ob_item:', t_struct.ob_item)

t: (1.0, 2.0)
item ids: (140379098725616, 140379098725832)
ob_size: 2
ob_item: <__main__.c_void_p_Array_2 object at 0x7fac8c24f6a8>


In [3]:
for ref in t_struct.ob_item:
    i_struct = PyFloatObject.from_address(ref)
    print(ref, hex(ref), '→', i_struct.ob_val)

140379098725616 0x7fac8e503cf0 → 1.0
140379098725832 0x7fac8e503dc8 → 2.0


## Inspecting an array of `c_double`

`Modules/arraymodule.c`

```c
struct arraydescr {
    char typecode;
    int itemsize;
    PyObject * (*getitem)(struct arrayobject *, Py_ssize_t);
    int (*setitem)(struct arrayobject *, Py_ssize_t, PyObject *);
    int (*compareitems)(const void *, const void *, Py_ssize_t);
    const char *formats;
    int is_integer_type;
    int is_signed;
};

typedef struct arrayobject {
    PyObject_VAR_HEAD
    char *ob_item;
    Py_ssize_t allocated;
    const struct arraydescr *ob_descr;
    PyObject *weakreflist; /* List of weak references */
    int ob_exports;  /* Number of exported buffers */
} arrayobject;

/* ... */

static PyObject *
d_getitem(arrayobject *ap, Py_ssize_t i)
{
    return PyFloat_FromDouble(((double *)ap->ob_item)[i]);
}

static int
d_setitem(arrayobject *ap, Py_ssize_t i, PyObject *v)
{
    double x;
    if (!PyArg_Parse(v, "d;array item must be float", &x))
        return -1;
    if (i >= 0)
                 ((double *)ap->ob_item)[i] = x;
    return 0;
}
```

In [4]:
import array

a = array.array('d', (30.0, 40.0))

ITEMS = len(a)

class PyArrayObject(ctypes.Structure):
    _fields_ = [("ob_refcnt", ctypes.c_long),
                ("ob_type", ctypes.c_void_p),
                ("ob_size", ctypes.c_ulong),
                ("ob_item", ctypes.c_void_p * ITEMS),
                ("allocated", ctypes.c_ulong),
                ("ob_descr", ctypes.c_void_p),
                ("weakreflist", ctypes.c_void_p),
                ("ob_exports", ctypes.c_int),
               ]

a_struct = PyArrayObject.from_address(id(a))

print('ob_size:', a_struct.ob_size)
print('ob_item:', a_struct.ob_item)

ob_size: 2
ob_item: <__main__.c_void_p_Array_2 object at 0x7fac8c24fae8>


In [5]:
for val in a_struct.ob_item:
    print(val, ctypes.cast(val, ctypes.POINTER(ctypes.c_double)))

140379089740144 <__main__.LP_c_double object at 0x7fac8c24f510>
2 <__main__.LP_c_double object at 0x7fac8c24f510>
